<p><font size="6" color='grey'> <b>

Generative KI. Verstehen. Anwenden. Gestalten.
</b></font> </br></p>

<p><font size="5" color='grey'> <b>
OutputParser
</b></font> </br></p>


---

In [ ]:
#@title 🔧 Umgebung einrichten{ display-mode: "form" }
!uv pip install --system -q git+https://github.com/ralf-42/GenAI.git#subdirectory=04_modul
from genai_lib.utilities import (
    check_environment,
    get_ipinfo,
    setup_api_keys,
    mprint,
    install_packages,
    mermaid,
    get_model_profile,
    extract_thinking,
    load_chat_prompt_template
)
setup_api_keys(['OPENAI_API_KEY'], create_globals=False)
print()
check_environment()
print()
get_ipinfo()

<p><font color='black' size="5">
⏸️ 5-Minuten-Check:
</font></p>

**Ziel:** Prüfen, ob du das vorherige Kapitel verstanden hast – nicht, ob es gerade läuft.

**Aufgabe** (5 Minuten, ohne Vorlage):

Rekonstruiere die zentrale Idee oder Code-Struktur des letzten Abschnitts selbstständig
(kein Copy & Paste, kein Nachschlagen).

Wähle eine der folgenden Optionen:

+ Erkläre in 1–2 Sätzen, was hier konzeptionell passiert.

+ Verändere eine Kleinigkeit (z. B. Prompt, Parameter, Reihenfolge) und beschreibe die Auswirkung.

+ Markiere eine Stelle, die du nicht sicher erklären kannst, und formuliere eine konkrete Frage dazu.

**Hinweis:**
Nicht alles muss „fertig“ oder „korrekt“ sein. Entscheidend ist, wo dein Verständnis gerade endet

# 1 | Intro
---

LangChain stellt eine Vielzahl von Ausgabeparsern bereit, die speziell darauf ausgelegt sind, Informationen aus den Ergebnissen großer Sprachmodelle (LLMs) effizient zu extrahieren und zu strukturieren. Diese Parser sind essenzielle Bestandteile der LangChain-Architektur und häufig zentrale Elemente sogenannter LangChain-Ketten. Solche Ketten bestehen aus konfigurierbaren Abfolgen von Operationen, die Modellausgaben verarbeiten und für weiterführende Anwendungen nutzbar machen.


**Warum sind OutputParser so wichtig?**
+ LLMs geben standardmäßig unstrukturierte Texte zurück.
+ OutputParser sind nötig, um das LLM-Output strukturiert weiterzuverarbeiten.
+ Besonders bei komplexen Anwendungen (z. B. Ketten mit mehreren Modellen, Agenten oder RAG-Systemen) müssen die Antworten klar definiert sein.

**`with_structured_output()` als State-of-the-Art (LangChain 1.0+):**
+ Die Methode **`with_structured_output()`** ist die moderne Lösung für strukturierte Ausgaben seit LangChain 1.0+.
+ Nutzt die native **OpenAI Structured Output API**, die Schema-Konformität auf Modell-Ebene garantiert.
+ **Garantierte Zuverlässigkeit** durch automatisches Retry bei Schema-Verletzungen.
+ **Einfacher Code** ohne manuelle Format-Instruktionen im Prompt.
+ **Production-Ready** - Teil der stabilen LangChain 1.0+ API.

<p><font color='black' size="5">
Parser-Kategorien und ihr Status:
</font></p>

| Parser-Typ | Beispiele | Status | Verwendung |
|-----------|-----------|--------|------------|
| **Einfache Parser** | `StrOutputParser`, `JsonOutputParser` | ✅ **Essentiell** | Standard in LCEL-Chains |
| **Strukturierte Parser** | `PydanticOutputParser`, `XMLOutputParser` | ⚠️ **Größtenteils ersetzt** | Durch `with_structured_output()` |
| **Spezial-Parser** | `OutputFixingParser`, `RetryParser` | 🔧 **Nische** | Fehlerbehandlung, Legacy |

Die Methode with_structured_output() wird durch mehrere Anbieter unterstützt ...



| Anbieter  | Modelle mit Support                                         |
|-----------|------------------------------------------------------------|
| OpenAI    | GPT-4, GPT-4o, GPT-3.5 mit [translate:Function Calling/JSON]       |
| Mistral   | Mistral Small, weitere Mistral Modelle                      |
| Google    | VertexAI LLMs                                              |
| Anthropic | Claude Modelle mit [translate:Tool-/Funktionsschnittstellen]       |


# 2 | OutputParser (einfach)
---

<p><font color='black' size="5">
Basismodell
</font></p>

In [ ]:
# Importe LangChain 1.0+
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field

In [ ]:
# Modell (Kurznotation: "provider:model")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)

<p><font color='blue' size="4">
Ohne Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
input = "Was ist Machine Learning?"
response = llm.invoke(input)

In [ ]:
for r in response:
    print(r)

<p><font color='blue' size="4">
Mit Nachbearbeitung mit einem Parser
</font></p>

In [ ]:
parser = StrOutputParser()
chain = llm | parser
response = chain.invoke(input)

# 3 | Datenformat Parser
---

LangChain bietet eine breite Palette an Parsern, die unterschiedliche Datenformate verarbeiten können, was seine Einsatzmöglichkeiten erheblich erweitert. Es unterstützt unter anderem die nahtlose Integration von Pandas-Datenrahmen, kommagetrennten Listen, JSON-Strukturen sowie Datums- und Zeitobjekten. Diese Flexibilität ermöglicht eine effiziente Anpassung an verschiedene Arten von Dateneingaben und macht LangChain zu einem leistungsstarken Werkzeug für die Analyse und Verarbeitung von Daten. Im Folgenden werden einige dieser Parser genauer betrachtet, ihre praktischen Anwendungen demonstriert und aufgezeigt, wie sie zur Optimierung von Prozessen und zur Gewinnung wertvoller Erkenntnisse beitragen können.

**Beispiel:** Die Ausgabe des LLM kann in einem JSON-Format strukturiert werden.

In [ ]:
#
# Variante: Prompt
#
prompt = (
    "Bitte gib mir eine fiktive Person mit Name und Alter im JSON-Format zurück, z.B. "
    '{"name": "Max", "age": 30}'
)

response = llm.invoke(prompt)

mprint("### 🤖 KI:")
mprint("---")
mprint(response.content)

In [ ]:
#
# Parser: JsonOutputParser
#
parser = JsonOutputParser()

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative Assistant."),
    ("human", "Give name and age of a person in JSON format."),
])

# Kette
chain = prompt | llm | parser

# Ausführen
response = chain.invoke({})

mprint("### 🤖 KI:")
mprint("---")
print(response)

# 4 | Structured Output (mittel)

Der **Structured Output** Ansatz in LangChain mit der Methode `with_structured_output()` ist die **state-of-the-art Lösung** für strukturierte LLM-Ausgaben in **LangChain**. Diese Methode nutzt die native Structured Output API von OpenAI, die garantiert, dass die Ausgabe dem definierten Schema entspricht.

**Vorteile von `with_structured_output()`:**
- **Garantierte Schema-Konformität**: OpenAI API erzwingt die Struktur auf Modell-Ebene
- **Kein weiterer Parser nötig**: Direkter Rückgabewert als Pydantic-Objekt
- **Höhere Zuverlässigkeit**: Automatisches Retry bei Schema-Verletzungen
- **Bessere Effizienz**: Keine zusätzlichen Format-Instruktionen im Prompt
- **Typ-Sicherheit**: Vollständige IDE-Unterstützung und Type-Hints
- **Production-Ready**: Teil der stabilen LangChain 1.0+ API
- **Best Practice**: Empfohlener Ansatz für alle strukturierten Outputs

Der erste Schritt besteht in der Definition eines **Pydantic BaseModel**, das die Struktur der erwarteten Ausgabe definiert. Anschließend wird das LLM mit `with_structured_output()` konfiguriert.

In [ ]:
#@title 🧜‍♀️ Prozess-Diagramm { display-mode: "form" }

diagram = """

flowchart TD
    A["LLM.with_structured_output(PydanticModel)"] --> B["Konfiguriertes LLM"]
    B --> C["Prompt senden"]
    C --> D["LLM generiert strukturierten Output"]
    D --> E["Parsing + Validierung"]
    E --> F["Pydantic-Objekt"]
    F --> G["Weiterverarbeitung"]

"""
mermaid(diagram, width=500, height=600)

Pydantic ist eine Python-Bibliothek, die automatisch Daten validiert und konvertiert, indem sie Typ-Hinweise nutzt und Datenstrukturen deklarativ mit Python-Klassen zu definieren.

In [ ]:
# Pydantic-Modell definieren
class Response(BaseModel):
    user_input: str = Field(description="Frage des Benutzers")
    response: str = Field(description="Antwort auf die Frage des Benutzers")
    source: str = Field(description="Verwendete Quelle (Website) für die Antwort")

print(Response.model_json_schema())

In [ ]:
# LLM mit strukturiertem Output konfigurieren. Kein Parser mehr nötig!
structured_llm = llm.with_structured_output(Response)

In [ ]:
# System-Prompt
system_prompt = """
Beantworte die folgende Frage so präzise wie möglich.
Gib auch die Quelle deiner Information an.
"""

# Chat-Prompt-Template mit externen Variablen
prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("human", "{user_input}")
])

user_input = "Was ist Machine Learning?"

In [ ]:
# Chain erstellen - kein Parser mehr nötig!
chain = prompt | structured_llm

In [ ]:
# Funktion zur Aufbereitung der Druck-Ausgabe
def markdown_out(response):
    mprint(f"### 🧑 Mensch")
    mprint(f"{response.user_input}")
    mprint(f"### 🤖 KI:")
    mprint(f"{response.response}")
    mprint(f"### Quelle:\n {response.source}")

In [ ]:
# Aufruf Chain mit Dictionary (ChatPromptTemplate)
parameter = {}
parameter["system_prompt"] = system_prompt
parameter["user_input"] = user_input
response = chain.invoke(parameter)

markdown_out(response)

<p><font color='black' size="5">
Datum/Uhrzeit
</font></p>

Durch die Verwendung von Pydantics `datetime` Typ wird automatisch eine Validierung durchgeführt und das Datum in ein standardisiertes Python `datetime`-Objekt umgewandelt.

Dies ist besonders hilfreich für Anwendungen wie Terminplanung, Datenanalyse oder andere Bereiche, in denen eine präzise Verarbeitung zeitlicher Informationen erforderlich ist. Der moderne Ansatz mit `with_structured_output()` ist zuverlässiger als der veraltete `DatetimeOutputParser` und nutzt die garantierte Schema-Konformität der OpenAI API.

In [ ]:
# Importe für Datum/Uhrzeit
from datetime import datetime

In [ ]:
# Pydantic-Modell für Datum/Uhrzeit (LangChain 1.0+ Ansatz)
class DateResponse(BaseModel):
    date: datetime = Field(description="Das extrahierte Datum als datetime-Objekt")
    explanation: str = Field(description="Kurze Erklärung zum Datum")

# Prompt-Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Beantworte die Frage und gib das Datum zurück. Wenn du das Datum nicht kennst, verwende 1111-01-01."),
    ("human", "{user_input}")
])

Wir erstellen die Chain mit `with_structured_output()` - kein Parser mehr nötig!

In [ ]:
# LLM mit strukturiertem Output
structured_llm = llm.with_structured_output(DateResponse)

In [ ]:
# Chain ohne Parser!
chain = prompt | structured_llm

Wir werden nach zwei Daten fragen, einem realen und einem fiktiven.

In [ ]:
parameter = {}
parameter["user_input"] = "Wann wurde die Programmiersprache Python eingeführt?"
response = chain.invoke(parameter)

# Ergebnis: 20.02.1991 (oder 1991-02-20 als datetime)
print(f"Datum: {response.date}")
print(f"Erklärung: {response.explanation}")

In [ ]:
parameter = {}
parameter["user_input"] = "An welchem Tag startet der Große Krieg im Videospiel Fallout?"
response = chain.invoke(parameter)

# Ergebnis: 23.10.2077 (oder 2077-10-23 als datetime)
print(f"Datum: {response.date}")
print(f"Erklärung: {response.explanation}")

# 5 | Structured Output (komplex)
---

Nun wird ein Beispiel mit einer größeren Anzahl an Werten getestet. Das folgende Programm nimmt Texte in beliebigen Sprachen entgegen und übersetzt sie automatisch ins Deutsche, Englische, Spanische und Chinesische. Dank `with_structured_output()` ist die Struktur garantiert - ohne zusätzliche Format-Instruktionen.

**Schritt 1:** Definiere ein **Pydantic-Modell** für die erwartete Ausgabe. Mit `with_structured_output()` wird die Struktur garantiert.

In [ ]:
# Pydantic-Modell definieren
class TranslationResponse(BaseModel):
    detected: str = Field(description="Die Sprache der Nutzer-Eingabe")
    german: str = Field(description="German translation")
    english: str = Field(description="English translation")
    spanish: str = Field(description="Spanish translation")
    chinese: str = Field(description="Chinese translation")

**Schritt 2:** Erstelle einen **einfachen Prompt** - keine Format-Instruktionen mehr nötig!

In [ ]:
# ChatPromptTemplate
system_prompt = """
Übersetze in die angegebenen Sprachen.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", "{system_prompt}"),
    ("human", "{user_input}")
])

**Schritt 3:** Chain ohne Parser - das strukturierte LLM gibt direkt ein typsicheres Pydantic-Objekt zurück!

In [ ]:
# LLM mit strukturiertem Output konfigurieren (Kurznotation: "provider:model")
llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)
structured_llm = llm.with_structured_output(TranslationResponse)

In [ ]:
# Chain - kein Parser mehr nötig!
chain = prompt | structured_llm

Zunächst wird ein deutscher Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
# Beispiel 1: Deutscher Satz
parameter = {}
parameter["system_prompt"] = system_prompt
parameter["user_input"] = "Wann wurde Python eingeführt?"

response = chain.invoke(parameter)

# Ausgabe über Pydantic-Attribute
print(f"{'detected':10s}:  {response.detected}")
print(f"{'german':10s}:  {response.german}")
print(f"{'english':10s}:  {response.english}")
print(f"{'spanish':10s}:  {response.spanish}")
print(f"{'chinese':10s}:  {response.chinese}")

Dann wird ein englischer Satz getestet, um zu beobachten, wie die Übersetzung in die drei Zielsprachen erfolgt.

In [ ]:
# Beispiel 2: Englischer Satz
parameter = {}
parameter["system_prompt"] = system_prompt
parameter["user_input"] = "Who rides so late through night and wind?"

response = chain.invoke(parameter)

# Ausgabe über Pydantic-Attribute
print(f"{'detected':10s}:  {response.detected}")
print(f"{'german':10s}:  {response.german}")
print(f"{'english':10s}:  {response.english}")
print(f"{'spanish':10s}:  {response.spanish}")
print(f"{'chinese':10s}:  {response.chinese}")

# 6 | Eigene Parser
---

In bestimmten Szenarien kann es sinnvoll sein, einen benutzerdefinierten Parser zu erstellen, um die Modellausgabe eindeutig zu formatieren.  

Dafür bietet sich die Verwendung von **RunnableLambda** oder **RunnableGenerator** in LCEL an, was für viele Fälle ein guten Ansatz ist.  

<p><font color='black' size="5">
Anwendungsfall
</font></p>



Large Language Models (LLMs) wie GPT-4 können Text generieren, der Code und erklärende Beschreibungen nahtlos vermischt. Dies kann zwar für Lern- und Dokumentationszwecke unglaublich nützlich sein, kann aber eine Herausforderung darstellen, wenn aus solchen Ausgaben mit gemischtem Inhalt nur der Code extrahiert und ausgeführt werden muss. Um dies zu beheben, implementieren wir eine einfache Funktion, die nicht-Python-Codezeilen aus einer gegebenen Textzeichenfolge entfernt.

Bei diesem Ansatz werden reguläre Ausdrücke verwendet, um Zeilen zu identifizieren und beizubehalten, die der typischen Python-Syntax entsprechen, während Zeilen verworfen werden, die beschreibender Text zu sein scheinen. Aufgrund der inhärenten Komplexität und Variabilität sowohl von Python-Code als auch von natürlicher Sprache kann diese Methode jedoch nie perfekt sein. Sie basiert auf heuristischen Mustern, die Code manchmal fälschlicherweise als Text klassifizieren oder umgekehrt.

Im nächsten Abschnitt werden wir untersuchen, wie ein anderes LLM beim Entfernen von Nicht-Python-Code helfen kann und möglicherweise eine ausgefeiltere und genauere Lösung bietet. Das folgende Beispiel enthält eine Mischung aus LLM-Kommentaren und generiertem Code.









In [ ]:
from langchain_core.runnables import RunnableLambda

# Benutzer-Funktion zur Trennung von Erläuterung und Code
def parse_ai_message(ai_message):
    """Trennt die Erläuterung und den Code aus einer AIMessage und gibt sie separat zurück."""
    text = ai_message.content  # Extrahiere den reinen Textinhalt der AIMessage

    if "```" in text:
        # Trennen der Erläuterung und des Codes
        parts = text.split("```")
        explanation = parts[0].strip()
        code = parts[1].strip() if len(parts) > 1 else ""
    else:
        # Falls kein Codeblock vorhanden ist, geben wir nur die Erläuterung zurück
        explanation = text.strip()
        code = ""

    return explanation, code  # Rückgabe von zwei separaten Werten

In [ ]:
# RunnableLambda für das Parsing erstellen
parser = RunnableLambda(parse_ai_message)

In [ ]:
# Eingabe an die Pipeline senden und Ergebnis ausgeben
user_input = """
    Erkläre mir, wie man eine einfache Funktion in Python erstellt und gib ein Beispiel zur Berechnung von Primzahlen.
    Verwende bei Formeln das Format $ Formel $
"""

In [ ]:
# LLM und Parser verketten
chain = llm | parser

In [ ]:
explanation, code = chain.invoke(user_input)

In [ ]:
mprint("### 🤖 Erläuterung")
mprint(explanation)
mprint("### 🤖 Code")
print(code)

# A | Aufgabe
---

Die Aufgabestellungen unten bieten Anregungen, Sie können aber auch gerne eine andere Herausforderung angehen.

<p><font color='black' size="5">
Trennung Erläuterung/Code
</font></p>

Anstelle des eigenen Parsers wird `with_structured_output` verwendet.


<p><font color='black' size="5">
JSON-Parser mit LangChain
</font></p>


**Ziel:** Verständnis für den Einsatz von `JsonOutputParser` in LangChain.

**Hinweis:** Der `JsonOutputParser` ist für einfache JSON-Ausgaben nützlich, aber für strukturierte Daten ist `with_structured_output()` die bessere Wahl!

**Aufgabe:**  
1. Nutze den `JsonOutputParser` von LangChain, um eine KI-Antwort in JSON zu formatieren.  
2. Lasse ein Language Model (z. B. OpenAI GPT) eine Liste von drei zufälligen Städten in Deutschland generieren.  
3. Verwende den Parser, um die Ausgabe in ein JSON-Format umzuwandeln.

**Erwartete Ausgabe (Beispiel):**
```json
{
  "cities": ["Berlin", "Hamburg", "München"]
}
```



<p><font color='black' size="5">
Extraktion von Schlüsselwerten
</font></p>


**Ziel:** Nutzung von `with_structured_output()`, um strukturierte Daten aus natürlicher Sprache zu extrahieren.

**Aufgabe:**  
1. Definiere eine `Pydantic`-Datenklasse mit den Feldern: `name` (str), `alter` (int), `stadt` (str).  
2. Verwende `with_structured_output()` mit einem Language Model.  
3. Extrahiere aus einer gegebenen Beschreibung eine Person.

**Beispiel-Eingabe:**
> "Max Mustermann ist 35 Jahre alt und lebt in Berlin."

**Beispiel-Code:**
```python
class Person(BaseModel):
    name: str = Field(description="Name der Person")
    alter: int = Field(description="Alter in Jahren")
    stadt: str = Field(description="Wohnort")

llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)
structured_llm = llm.with_structured_output(Person)
```

**Erwartete Ausgabe (Pydantic-Modell):**
```python
Person(name="Max Mustermann", alter=35, stadt="Berlin")
```

Als JSON:
```json
{
  "name": "Max Mustermann",
  "alter": 35,
  "stadt": "Berlin"
}
```


<p><font color='black' size="5">
Parser für Listenformate
</font></p>


**Ziel:** Implementierung eines eigenen Parsers zur Umwandlung von KI-Ausgaben in Listen.

**Aufgabe:**  
1. Erstelle eine eigene Parser-Klasse, die eine durch Kommas getrennte Liste in ein Listenformat umwandelt.  
2. Verwende diesen Parser, um eine Liste von fünf beliebten Büchern aus einer Language-Model-Antwort zu extrahieren.

**Beispiel-Eingabe:**
> "Die Verwandlung, Faust, Der Prozess, Die Blechtrommel, Der Vorleser"

**Erwartete Ausgabe:**
```python
["Die Verwandlung", "Faust", "Der Prozess", "Die Blechtrommel", "Der Vorleser"]
```

<p><font color='black' size="5">
Kombination Parser & PromptTemplate
</font></p>


**Ziel:** Verwendung von `with_structured_output()` in Kombination mit `PromptTemplate`.

**Aufgabe:**  
1. Erstelle ein `PromptTemplate`, das eine strukturierte Antwort über ein Land ausgibt (Name, Hauptstadt, Einwohnerzahl).  
2. Verwende `with_structured_output()` mit einem Pydantic-Modell.

**Beispiel-Pydantic-Modell:**
```python
class CountryInfo(BaseModel):
    name: str = Field(description="Name des Landes")
    hauptstadt: str = Field(description="Hauptstadt des Landes")
    einwohner: int = Field(description="Einwohnerzahl")

llm = init_chat_model("openai:gpt-4o-mini", temperature=0.0)
structured_llm = llm.with_structured_output(CountryInfo)
```

**Beispiel-Prompt:**
> "Gib mir Informationen zu Frankreich."

**Erwartete Ausgabe:**
```python
CountryInfo(name="Frankreich", hauptstadt="Paris", einwohner=67000000)
```

Als JSON:
```json
{
  "name": "Frankreich",
  "hauptstadt": "Paris",
  "einwohner": 67000000
}
```

**Vorteil:** Keine manuelle Format-Instruktionen nötig - die API garantiert die Struktur!
